In [1]:
!pip install torch transformers datasets

import torch
from datasets import load_dataset
from transformers import pipeline

# Load the BoolQ dataset from Hugging Face.
# BoolQ is a Boolean (yes/no) QA task that includes a 'passage', 'question', and binary 'answer'.
boolq_dataset = load_dataset("boolq")

# Specify the model to use.
# "google/flan-t5-base" is an example instruction-tuned model; adjust the model_name as desired.
model_name = "google/flan-t5-base"

# Initialize the text-to-text generation pipeline.
# Use GPU if available.
device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)

def generate_answer(passage, question):
    """
    Formats the prompt with the passage and question, then uses the LLM to generate an answer.
    The prompt is structured so that the model is primed to answer in 'yes' or 'no'.
    """
    # Create a prompt that instructs the model about the task.
    prompt = f"Passage: {passage}\nQuestion: {question}\nAnswer:"
    # Generate the response; adjust max_length as needed.
    output = generator(prompt, max_length=16, do_sample=False)
    generated_text = output[0]['generated_text']
    return generated_text.strip().lower()

def parse_answer(text):
    """
    Parses the generated text to return a Boolean.
    The function checks for the presence of the words "yes" or "no" in the output.
    """
    if "yes" in text:
        return True
    elif "no" in text:
        return False
    else:
        # If the generated text does not clearly contain yes/no, you may choose to handle it differently.
        # Here we return None, and such examples could be skipped or treated as a wrong answer.
        return None

# Select the validation split of the BoolQ dataset for evaluation.
dataset = boolq_dataset["validation"]

predictions = []
labels = []

print("Evaluating BoolQ benchmark with LLM...")

# Loop over examples in the validation set.
for example in dataset:
    passage = example["passage"]
    question = example["question"]
    true_answer = example["answer"]  # Boolean (True/False)

    # Generate the answer using our LLM
    generated = generate_answer(passage, question)
    predicted = parse_answer(generated)

    # If the answer is ambiguous (i.e. parsing returns None), count it as incorrect.
    if predicted is None:
        predicted = False

    predictions.append(predicted)
    labels.append(true_answer)

# Calculate accuracy.
correct = sum(p == l for p, l in zip(predictions, labels))
accuracy = correct / len(labels)

print(f"Accuracy on the BoolQ validation set: {accuracy:.2%}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

README.md:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.69M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


Evaluating BoolQ benchmark with LLM...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Token indices sequence length is longer than the specified maximum sequence length for this model (826 > 512). Running this sequence through the model will result in indexing errors


Accuracy on the BoolQ validation set: 75.75%
